In [1]:
import threading
import cv2
import numpy as np
import tkinter as tk
from tkinter import Label
from PIL import Image, ImageTk
import pyttsx3
from keras.models import load_model
import os

In [2]:
model = load_model('emotion_model-099.keras')
engine = pyttsx3.init()

frame_count = 0
predict_interval = 5
current_emotion_label = "Neutral"
last_spoken = None

emoji_path = {
    0: 'C:/Users/HP/Downloads/Emoji/Angry.png',
    1: 'C:/Users/HP/Downloads/Emoji/Disgusted.png',
    2: 'C:/Users/HP/Downloads/Emoji/Fear.png',
    3: 'C:/Users/HP/Downloads/Emoji/Happy.png',
    4: 'C:/Users/HP/Downloads/Emoji/Neutral.png',
    5: 'C:/Users/HP/Downloads/Emoji/Sad.png',
    6: 'C:/Users/HP/Downloads/Emoji/Surprised.png'
}
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

face_cascade = cv2.CascadeClassifier(r'C:\Users\HP\Downloads\haarcascade_frontalface_default.xml')

In [3]:
def preprocess_face(face_img):
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (100, 100))
    resized = resized.astype('float32') / 255.0
    resized = np.reshape(resized, (1, 100, 100, 1))
    return resized

def overlay_emoji_on_frame(frame, emoji_img, x, y, w, h):
    emoji_resized = cv2.resize(emoji_img, (w, h))
    if emoji_resized.shape[2] == 4:
        emoji_rgb = emoji_resized[:, :, :3]  
        alpha_mask = emoji_resized[:, :, 3]  
        alpha_mask = alpha_mask / 255.0
        for c in range(0, 3):  
            frame[y:y + h, x:x + w, c] = (alpha_mask * emoji_rgb[:, :, c] + (1 - alpha_mask) * frame[y:y + h, x:x + w, c])
    else:
        frame[y:y + h, x:x + w] = cv2.addWeighted(frame[y:y + h, x:x + w], 0.5, emoji_resized, 0.5, 0)
    return frame

def predict_emotion_async(face):
    global current_emotion_label
    processed_face = preprocess_face(face)
    emotion_prediction = model.predict(processed_face)
    max_index = int(np.argmax(emotion_prediction))
    current_emotion_label = emotion_labels[max_index]

In [4]:
def update_frame():
    global frame_count, last_spoken, current_emotion_label

    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture video feed.")
        return
    
    frame = cv2.resize(frame, (320, 240))
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]

        if frame_count % predict_interval == 0:
            threading.Thread(target=predict_emotion_async, args=(face,)).start()

        cv2.putText(frame, current_emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        emoji_img_path = emoji_path.get(emotion_labels.index(current_emotion_label), None)
        if emoji_img_path and os.path.exists(emoji_img_path):
            emoji_img = cv2.imread(emoji_img_path, -1)
            frame = overlay_emoji_on_frame(frame, emoji_img, x, y, w, h)

        if current_emotion_label != last_spoken:
            last_spoken = current_emotion_label
            engine.say(current_emotion_label)
            engine.runAndWait()

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame)
    imgtk = ImageTk.PhotoImage(image=img)

    lblVideo.imgtk = imgtk
    lblVideo.configure(image=imgtk)

    frame_count += 1
    lblVideo.after(10, update_frame)

root = tk.Tk()
root.title("Emoji Generator from Facial Expression")
root.config(bg='khaki')
root.geometry('450x420')

heading = Label(root,text="Emoji Generator from\nFacial Expression",font=('Times New Roman', 24, 'bold'),justify="center",bg="khaki",fg="saddle brown")
heading.pack(pady=30) 

lblVideo = Label(root)
lblVideo.pack()

cap = cv2.VideoCapture(0)

update_frame()

root.mainloop()

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━